In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
blueblushed_hospital_dataset_for_practice_path = kagglehub.dataset_download('blueblushed/hospital-dataset-for-practice')
google_gemini_2_0_flash_api_api_gemini_2_0_flash_1_path = kagglehub.model_download('google/gemini-2.0-flash-api/Api/gemini-2.0-flash/1')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Installing libraries

In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

In [ ]:
!pip install langchain langchain-google-genai google-generativeai


### Import Libraries

In [ ]:
import pandas as pd
from kaggle_secrets import UserSecretsClient
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
from google.genai import types


import warnings
warnings.filterwarnings("ignore")


### Seting API Key

In [ ]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")

In [ ]:
import os
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai
from google.genai import types

### RAG Function

In [ ]:
# ----------------- 1. RAG Function ------------------
def generate_rag_response(user_text: str) -> str:
    client = genai.Client(api_key=secret_value_0)

    uploaded_file = client.files.upload(file="/kaggle/input/hospital-dataset-for-practice/hospital data analysis.csv")  # Adjust path if needed

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=uploaded_file.uri,
                    mime_type=uploaded_file.mime_type,
                ),
                types.Part.from_text(text=f"""You are a helpful assistant. Use the data in the file to answer this question directly and also note that the data which you present should be detailed. The user input is: {user_text}"""),
            ],
        ),
    ]

    config = types.GenerateContentConfig(response_mime_type="text/plain")

    try:
        rag_response = ""
        for chunk in client.models.generate_content_stream(
            model="gemini-2.0-flash-lite",
            contents=contents,
            config=config,
        ):
            rag_response += chunk.text
        return rag_response
    except genai.errors.ClientError as e:
        return "RAG failed to retrieve data."

In [ ]:
# ----------------- 2. Set up LangChain LLM ------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.7,
    verbose=True,
    api_key=secret_value_0
)

### Custom Tool or Function Calling

In [ ]:
# ----------------- 3. Define Tool for RAG ------------------
def rag_tool_func(query: str) -> str:
    return generate_rag_response(query)

rag_tool = Tool(
    name="RAGDataSearch",
    func=rag_tool_func,
    description="Use this tool to retrieve detailed hospital or organization data from a CSV based on user query."
)

# ----------------- 4. Agent Setup ------------------
tools = [rag_tool]

agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,handle_parsing_errors = True)

# Function to process the prompt with the agent
def process_prompt_with_agent(prompt):
    return agent.run(prompt)

### RAG & Agent Invocation with query

In [ ]:
# ----------------- 5. Run Agent ------------------
if __name__ == "__main__":
    user_input = "I am suffering from Appendicitis will I get cured?"  # Sample User Input
    # user_input = input("💬 Please enter your question: ")
    print("\n🧠 Agent reasoning and decision in progress...\n")
    # Call the rag_tool function to get the response before using it in the prompt
    response = rag_tool_func(user_input)
    text = user_input

    # Now define the prompt with 'text' and 'response' available
    # Used few shot prompting technique.
    prompt = f"""You are a helpful assistant to recommend a hospital or not for a user based on their records.
      You don't have any tools.
      You are given an input from the user: {text} and a response from RAG: {response}.
      The RAG response includes satisfaction scores and various columns based on a hospital or organization.
      Based on the RAG data, your job is to analyze it and say whether the user should take an appointment or not in that hospital.
      The ouput should be strictly be with a justification:
      Example 1:
      Yes, according to the data given you can reach out to the particular hospital or organisation.
      Example 2: No, according to the data given you cannot reach out to the particular hospital or organisation.
      Example 3: Not sure, according to the data it is difficult to understand whether to go to the particular hospital or organisation or not.
    """
    # Run the agent with the updated prompt that includes text and response
    output = agent.run(prompt)

    print("\n✅ Final Recommendation:\n", output)

#### Example 2

In [ ]:
# ----------------- 5. Run Agent ------------------
if __name__ == "__main__":
    user_input = "How many days should a person admit for if he is suffering from Fractured Leg?" # Sample User Input
    # user_input = input("💬 Please enter your question: ")
    print("\n🧠 Agent reasoning and decision in progress...\n")
    # Call the rag_tool function to get the response before using it in the prompt
    response = rag_tool_func(user_input)
    # Assign the user input to the 'text' variable
    text = user_input

    # Now define the prompt with 'text' and 'response' available
    # Used few shot prompting technique.
    prompt = f"""You are a helpful assistant to recommend a hospital or not for a user based on their records.
      You don't have any tools.
      You are given an input from the user: {text} and a response from RAG: {response}.
      The RAG response includes satisfaction scores and various columns based on a hospital or organization.
      Based on the RAG data, your job is to analyze it and say whether the user should take an appointment or not in that hospital.
      The ouput should be strictly be with a justification:
      Example 1:
      Yes, according to the data given you can reach out to the particular hospital or organisation.
      Example 2: No, according to the data given you cannot reach out to the particular hospital or organisation.
      Example 3: Not sure, according to the data it is difficult to understand whether to go to the particular hospital or organisation or not.
    """
    # Run the agent with the updated prompt that includes text and response
    output = agent.run(prompt)

    print("\n✅ Final Recommendation:\n", output)